# Logistic Regression Project


## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=3cceef43eccc3f880576706fc482f63b9539f79d40a65ad2a859767b78e9c999
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('logregconsult').getOrCreate()

In [4]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,
                     header=True)

In [5]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



### Check out the data

In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [7]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

### Format for MLlib

We'll ues the numerical columns. We'll include Account Manager because its easy enough, but keep in mind it probably won't be any sort of a signal because the agency mentioned its randomly assigned!

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [10]:
output = assembler.transform(data)

In [11]:
final_data = output.select('features','churn')

### Test Train Split

In [12]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

### Fit the model

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
lr_churn = LogisticRegression(labelCol='churn')

In [15]:
fitted_churn_model = lr_churn.fit(train_churn)

In [16]:
training_sum = fitted_churn_model.summary

In [17]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                641|                641|
|   mean|0.16536661466458658|0.12480499219968799|
| stddev| 0.3718012297534502|   0.33075576484022|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [19]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [20]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[27.0,8628.8,1.0,...|    0|[5.58153299098017...|[0.99624734879592...|       0.0|
|[28.0,8670.98,0.0...|    0|[8.12668852957653...|[0.99970454204963...|       0.0|
|[29.0,8688.17,1.0...|    1|[2.64702460901406...|[0.93382736690441...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.57230717602188...|[0.98977161145581...|       0.0|
|[29.0,10203.18,1....|    0|[3.83196461751587...|[0.97879249670201...|       0.0|
|[29.0,12711.15,0....|    0|[5.64557694103533...|[0.99647932949692...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.11384380118893...|[0.99779285044503...|       0.0|
|[30.0,8677.28,1.0...|    0|[4.20437375599875...|[0.98528949672285...|       0.0|
|[30.0,10744.14,1....|    1|[1.67132230803153...|[0.84175203952269...|       0.0|
|[31.0,5387.75,0

### Using AUC

In [21]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [22]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [23]:
auc

0.7675475687103593

### Predict on brand new unlabeled data

We still need to evaluate the new_customers.csv file!

In [24]:
final_lr_model = lr_churn.fit(final_data)

In [25]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)

In [26]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [27]:
test_new_customers = assembler.transform(new_customers)

In [28]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [29]:
final_results = final_lr_model.transform(test_new_customers)

In [30]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



Ok! That is it! Now we know that we should assign Acocunt Managers to Cannon-Benson,Barron-Robertson,Sexton-GOlden, and Parks-Robbins!